In [ ]:
import spacy
from spacy import displacy
from spacytextblob.spacytextblob import SpacyTextBlob

# Load spaCy model
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('spacytextblob')  # Add sentiment analysis

# Process a sample review
sample_review = df.iloc[0]["Text"]
doc = nlp(sample_review)

# 1. Named Entity Recognition (NER)
print("--- Named Entities (Products/Brands) ---")
for ent in doc.ents:
    if ent.label_ in ["ORG", "PRODUCT"]:  # Focus on brands/products
        print(f"{ent.text} ({ent.label_})")

# 2. Sentiment Analysis
print("\n--- Sentiment Analysis ---")
print(f"Review Text: {sample_review}")
print(f"Polarity: {doc._.polarity:.2f} (-1 to +1)")
print(f"Sentiment: {'Positive' if doc._.polarity > 0 else 'Negative'}")

# 3. Visualize NER
displacy.render(doc, style="ent", jupyter=True)

In [1]:
# Add custom sentiment rules
def custom_sentiment(doc):
    if "bitter" in doc.text.lower():
        doc._.polarity -= 0.3  # Adjust sentiment score
    return doc

nlp.add_pipe(custom_sentiment, last=True)

NameError: name 'nlp' is not defined

ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <function amazon_sentiment at 0x7c3054b81fc0> (name: 'amazon_sentiment').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.